In [16]:
import requests
import requests 
import json
import os
from dotenv import load_dotenv
import pandas as pd
from pandas import json_normalize
import numpy as np
import sys
sys.path.append('src')
import apilast as als
import sqltools as sqt
import cleansing as cln
import datetime

In [17]:
load_dotenv()

True

In [3]:
keylast = os.getenv("keylast")
l_user = 'sinatxester'
limit = 1000


In [ ]:
url = f'http://ws.audioscrobbler.com/2.0/?method=user.getrecenttracks&user={l_user}&limit={limit}&api_key={keylast}&format=json'

In [ ]:
req = requests.get(url).json()['recenttracks']

In [ ]:
pages = int(req['@attr']['totalPages'])

In [ ]:
pages

In [ ]:
listas = []
for i in range(1,pages+1):
    page = i
    url_t = f'http://ws.audioscrobbler.com/2.0/?method=user.getrecenttracks&user={l_user}&limit={limit}&page={page}&api_key={keylast}&format=json'
    req_t = requests.get(url_t).json()['recenttracks']['track']
    listas.append(req_t)
    if i%5==0:
        print(f'pag {i} done')
tracks = [l for lista in listas for l in lista]
sinatxe = als.df_fromtracks(tracks)

In [ ]:
sinatxe.sample(5)

In [ ]:
sinatxe.shape

In [4]:
sinatxe_tot = als.tot_scro(l_user)

pag 5 done
pag 10 done
pag 15 done
pag 20 done
pag 25 done
pag 30 done
pag 35 done
pag 40 done
pag 45 done
pag 50 done
pag 55 done
pag 60 done
pag 65 done
pag 70 done
pag 75 done


In [34]:
sinatxe_tot.head(8)

,uts,artist,artist_mbid,album,album_mbid,title,track_mbid,fechahora
0,1642353859,Tori Amos,c0b2500e-0cef-4130-869d-732b23ed9df5,Strange Little Girls,08da7d96-c961-483f-bf35-904efb50c748,I Don't Like Mondays,0f552f8a-e073-41b9-81bb-1a6c8f22a50b,2022-01-16 18:24:19
1,1642353589,R.E.M.,ea4dfa26-f633-4da6-a52a-f49ea4897b58,Out of Time,04635060-f2e5-49d7-88c7-39faf09f5c48,Losing My Religion,0051b6be-bb2a-3c06-b47d-953eff1a7eee,2022-01-16 18:19:49
2,1642350684,Lana Del Rey,b7539c32-53e7-4908-bda3-81449c367da6,Blue Banisters,04ffa6bb-db29-4243-9c71-b32b79c9b0ec,Violets for Roses,14befa8f-d415-498f-9d9b-e4381b495626,2022-01-16 17:31:24
3,1642152224,Bebe,789b4794-71a2-44fd-86b4-baa8ed6755ed,Pafuera Telarañas,33af452e-a56f-413a-b159-156b131fd5c0,Con Mis Manos,3c6461e7-2c6e-48e5-8d8b-e2ae9e644092,2022-01-14 10:23:44
4,1642152101,Nick Drake,99ea432a-e3d8-42cb-9d5e-db316a6a8458,Pink Moon,278903c5-465e-4dfe-b278-10e7ce02f0d3,Pink Moon,075c79dc-348e-333e-a993-ad8cbea78699,2022-01-14 10:21:41
5,1642151899,Amaral,7a519099-d4c8-46c8-805b-214dab0e58a2,Estrella de mar,07906a09-22dc-4243-ae57-624720962e6b,¿Qué será?,4cd7dd2d-555b-3261-a6b1-65499edabb5c,2022-01-14 10:18:19
6,1642151125,Peret,796481a3-2977-45b6-84b3-eb06bb10a903,De Los Cobardes Nunca Se Ha Escrito Nada,e6fba579-d036-4ceb-b522-ffe0a577e8d2,El muerto vivo,015613b4-21da-40e2-86a3-24cd4e64615a,2022-01-14 10:05:25
7,1642150848,Bob Dylan,72c536dc-7137-4477-a521-567eeb840fa8,Blonde on Blonde,09bb38fb-60d5-3e53-9333-d8a815761383,Rainy Day Women #12 & 35,02b32b23-cd89-48b5-b371-ddbf4942004a,2022-01-14 10:00:48


In [23]:
print(
    datetime.datetime.fromtimestamp(
        int("1284105682")
    ).strftime('%Y-%m-%d %H:%M:%S'))

2010-09-10 10:01:22


In [21]:
# for i,r in sinatxe_tot.iterrows():
#     print(datetime.datetime.fromtimestamp(int(r.uts)).strftime('%Y-%m-%d %H:%M:%S'))

In [24]:
def utslocal (uts):
    '''
    recibe una cadena de texto o interger que es uts(unix time stamp, valor en segundos de una fecha)
        unix timestamp count = This count starts at the Unix Epoch on January 1st, 1970 at UTC
    devuelve fecha y hora local. 
    '''
    if type(uts) == int:
        return datetime.datetime.fromtimestamp(uts).strftime('%Y-%m-%d %H:%M:%S')
    else:
        return datetime.datetime.fromtimestamp(int(uts)).strftime('%Y-%m-%d %H:%M:%S')

In [26]:
sinatxe_tot['fechahora'] = sinatxe_tot.uts.apply(utslocal)

In [29]:
sinatxe_tot.head(3)

,uts,artist,artist_mbid,album,album_mbid,title,track_mbid,fechahora
0,1642353859,Tori Amos,c0b2500e-0cef-4130-869d-732b23ed9df5,Strange Little Girls,08da7d96-c961-483f-bf35-904efb50c748,I Don't Like Mondays,0f552f8a-e073-41b9-81bb-1a6c8f22a50b,2022-01-16 18:24:19
1,1642353589,R.E.M.,ea4dfa26-f633-4da6-a52a-f49ea4897b58,Out of Time,04635060-f2e5-49d7-88c7-39faf09f5c48,Losing My Religion,0051b6be-bb2a-3c06-b47d-953eff1a7eee,2022-01-16 18:19:49
2,1642350684,Lana Del Rey,b7539c32-53e7-4908-bda3-81449c367da6,Blue Banisters,04ffa6bb-db29-4243-9c71-b32b79c9b0ec,Violets for Roses,14befa8f-d415-498f-9d9b-e4381b495626,2022-01-16 17:31:24


In [32]:
sinatxe_tot['fechahora'] = pd.to_datetime(sinatxe_tot['fechahora'])

In [33]:
sinatxe_tot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74127 entries, 0 to 74126
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   uts          74127 non-null  object        
 1   artist       74127 non-null  object        
 2   artist_mbid  74127 non-null  object        
 3   album        74127 non-null  object        
 4   album_mbid   74127 non-null  object        
 5   title        74127 non-null  object        
 6   track_mbid   74127 non-null  object        
 7   fechahora    74127 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(7)
memory usage: 4.5+ MB


In [14]:
sinatxe_tot[sinatxe_tot['artist']=='Lana Del Rey']['artist_mbid'].value_counts()

b7539c32-53e7-4908-bda3-81449c367da6    1245
                                           5
Name: artist_mbid, dtype: int64

In [35]:
sinatxe_tot[sinatxe_tot['artist_mbid']=='']

,uts,artist,artist_mbid,album,album_mbid,title,track_mbid,fechahora
61,1641267473,Rocío Márquez,,Firmamento,,Tierra Y Centro,,2022-01-04 04:37:53
66,1641266553,Dani Flaco,,Versos y madera,,Canción de cuna,,2022-01-04 04:22:33
68,1641266182,Robe.,,Lo que aletea en nuestras cabezas,091ad116-ea96-4413-879c-75c530ca39bb,Ruptura Leve,704fa388-740e-4a54-bdba-03ca77374a9a,2022-01-04 04:16:22
72,1641265051,Luis Ramiro,,Inéditas,,Tras la pared,,2022-01-04 03:57:31
75,1641264416,Rosario La Tremendita,,A tiempo,a9a372b6-27dc-4524-8c07-ef68318d4bbe,Enriqueta La Pescaera (Bulería),,2022-01-04 03:46:56
...,...,...,...,...,...,...,...,...
74121,1420143099,Marea,,Jauría De Perros Verdes (Rarezas),,Marea (Versión Acústica),,2015-01-01 21:11:39
74123,1420142462,Oasis,,(What's the Story) Morning Glory?,031daf2e-ee11-4a34-8a90-105f76be0729,She's Electric,0093e8c3-bfb3-3eda-98c3-233a514c1d71,2015-01-01 21:01:02
74124,1420142122,Oasis,,(What's the Story) Morning Glory?,031daf2e-ee11-4a34-8a90-105f76be0729,Don't Look Back in Anger,00627e23-d3e1-4efb-8506-6e9fea811eaf,2015-01-01 20:55:22
74125,1420141740,Oasis,,(What's the Story) Morning Glory?,031daf2e-ee11-4a34-8a90-105f76be0729,Don't Look Back in Anger,00627e23-d3e1-4efb-8506-6e9fea811eaf,2015-01-01 20:49:00
